In [ ]:
!sudo add-apt-repository -y ppa:alessandro-strada/ppa 2>&1 > /dev/null
!sudo apt-get update -qq 2>&1 > /dev/null
!sudo apt -y install -qq google-drive-ocamlfuse 2>&1 > /dev/null
!google-drive-ocamlfuse

!sudo apt-get install -qq w3m # to act as web browser 
!xdg-settings set default-web-browser w3m.desktop # to set default browser
%cd /content
!mkdir drive
%cd drive
!mkdir MyDrive
%cd ..
!google-drive-ocamlfuse /content/drive/MyDrive



debconf: unable to initialize frontend: Dialog
debconf: (No usable dialog-like program is installed, so the dialog based frontend cannot be used. at /usr/share/perl5/Debconf/FrontEnd/Dialog.pm line 76, <> line 1.)
debconf: falling back to frontend: Readline
debconf: unable to initialize frontend: Readline
debconf: (This frontend requires a controlling tty.)
debconf: falling back to frontend: Teletype
dpkg-preconfigure: unable to re-open stdin: 
/usr/bin/xdg-open: 851: /usr/bin/xdg-open: www-browser: not found
/usr/bin/xdg-open: 851: /usr/bin/xdg-open: links2: not found
/usr/bin/xdg-open: 851: /usr/bin/xdg-open: elinks: not found
/usr/bin/xdg-open: 851: /usr/bin/xdg-open: links: not found
/usr/bin/xdg-open: 851: /usr/bin/xdg-open: lynx: not found
/usr/bin/xdg-open: 851: /usr/bin/xdg-open: w3m: not found
xdg-open: no method available for opening 'https://accounts.google.com/o/oauth2/auth?client_id=564921029129.apps.googleusercontent.com&redirect_uri=https%3A%2F%2Fgd-ocaml-auth.appspot.

In [ ]:
% cd /content

/content


In [ ]:
import json
path = "/content/drive/MyDrive/IDL-Project/WebQA_data_first_release/WebQA_train_val.json"
q = json.load(open(path,'r'))


In [ ]:
q.pop(list(q.keys())[0])

{'A': ['"Yes, both the National Museum of the American Indian in Washington, D.C. and the Xanadu House in Kissimmee, Florida are beige."'],
 'Guid': 'd5bbc6d80dba11ecb1e81171463288e9',
 'Q': '"Are both the National Museum of the American Indian in Washington, D.C. and the Xanadu House in Kissimmee, Florida the same color?"',
 'Qcate': 'YesNo',
 'img_negFacts': [{'caption': 'National Museum of the American Indian',
   'image_id': 30095223,
   'imgUrl': 'https://upload.wikimedia.org/wikipedia/commons/thumb/2/25/National_Museum_of_the_American_Indian.jpg/450px-National_Museum_of_the_American_Indian.jpg',
   'title': 'National Museum of the American Indian',
   'url': 'https://commons.wikimedia.org/wiki/File:National_Museum_of_the_American_Indian.jpg'},
  {'caption': 'Xanadu - futuristic architecture Xanadu House Interior',
   'image_id': 30388945,
   'imgUrl': 'https://upload.wikimedia.org/wikipedia/commons/thumb/1/12/Xanadu_-_futuristic_architecture.jpg/800px-Xanadu_-_futuristic_architec

In [ ]:
import torch
import torch.nn as nn
import json
# import clip
from torch.utils.data import DataLoader, Dataset
%pip install transformers
from transformers import BertTokenizer
from tqdm import tqdm


class QADataset(Dataset):
  def __init__(self,  data):
    self.data = data
    self.tokenizer = BertTokenizer.from_pretrained('bert-base-cased')
    self.instances = list(self.data.keys())
    self.Qs = [self.tokenizer(self.data[instance]['Q'], 
                               padding='max_length', max_length = 512, truncation=True,
                                return_tensors="pt") for instance in self.instances]
    self.lens = [(Q['input_ids'] == 0).nonzero(as_tuple=False)[0][1].numpy() for Q in self.Qs]
    self.modals = [0 if len(self.data[instance]['img_posFacts']) else 1 for instance in self.instances]

  
  def __len__(self):
    return len(self.instances)
  
  def __getitem__(self, idx):
    return self.Qs[idx], self.lens[idx], self.modals[idx]

     |████████████████████████████████| 4.0 MB 4.8 MB/s 
     |████████████████████████████████| 895 kB 41.4 MB/s 
     |████████████████████████████████| 6.6 MB 41.5 MB/s 
     |████████████████████████████████| 596 kB 39.7 MB/s 
     |████████████████████████████████| 77 kB 6.5 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [ ]:
import json
train_path = "/content/drive/MyDrive/IDL-Project/WebQA_data_first_release/WebQA_train_val.json"
q = json.load(open(train_path,'r'))
# test_path = "/content/drive/MyDrive/18-786-Intro-DL/IDL-Project/WebQA_data_first_release/WebQA_test.json"
# qq =  json.load(open(test_path,'r'))

val_pts = {k:v for k,v in q.items() if v['split'] == 'val'}
train_pts = {k:v for k,v in q.items() if v['split'] == 'train'}

train_loader = DataLoader(QADataset(train_pts),batch_size=128,shuffle=True)
val_loader = DataLoader(QADataset(val_pts),batch_size=128,shuffle=False)
# test_loader = DataLoader(QADataset(qq),batch_size=128,shuffle=False)

Downloading:   0%|          | 0.00/208k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/29.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/570 [00:00<?, ?B/s]

In [ ]:
next(iter(train_loader))

[{'attention_mask': tensor([[[1, 1, 1,  ..., 0, 0, 0]],
  
          [[1, 1, 1,  ..., 0, 0, 0]],
  
          [[1, 1, 1,  ..., 0, 0, 0]],
  
          ...,
  
          [[1, 1, 1,  ..., 0, 0, 0]],
  
          [[1, 1, 1,  ..., 0, 0, 0]],
  
          [[1, 1, 1,  ..., 0, 0, 0]]]),
  'input_ids': tensor([[[ 101,  107, 1731,  ...,    0,    0,    0]],
  
          [[ 101,  107, 2825,  ...,    0,    0,    0]],
  
          [[ 101, 1327, 2747,  ...,    0,    0,    0]],
  
          ...,
  
          [[ 101, 1109, 1148,  ...,    0,    0,    0]],
  
          [[ 101,  107, 1327,  ...,    0,    0,    0]],
  
          [[ 101, 1327, 1583,  ...,    0,    0,    0]]]),
  'token_type_ids': tensor([[[0, 0, 0,  ..., 0, 0, 0]],
  
          [[0, 0, 0,  ..., 0, 0, 0]],
  
          [[0, 0, 0,  ..., 0, 0, 0]],
  
          ...,
  
          [[0, 0, 0,  ..., 0, 0, 0]],
  
          [[0, 0, 0,  ..., 0, 0, 0]],
  
          [[0, 0, 0,  ..., 0, 0, 0]]])},
 tensor([57, 19, 20, 32, 37, 13, 21, 17, 31, 33, 22, 

In [ ]:
torch.cuda.empty_cache() # Use this often
!nvidia-smi

Thu Apr 14 18:30:12 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla K80           Off  | 00000000:00:04.0 Off |                    0 |
| N/A   40C    P8    25W / 149W |      0MiB / 11441MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
from torch import nn
from transformers import BertModel, BertConfig
from torch.nn.utils.rnn import pad_sequence, pack_padded_sequence, pad_packed_sequence

configuration = BertConfig('bert-base-cased')
bert = BertModel.from_pretrained('bert-base-cased').cuda().eval()

class BertClassifier(nn.Module):

    def __init__(self, dropout=0.5):

        super(BertClassifier, self).__init__()
        
        # print(configuration.hidden_size)
        self.bert = BertModel.from_pretrained('bert-base-cased')
        self.dropout = nn.Dropout(dropout)
        self.linear = nn.Linear(768, 2)

    def forward(self, seq_output, mask):
        _, pooled_output = self.bert(input_ids=seq_output, attention_mask=mask,return_dict=False)
        dropout_output = self.dropout(pooled_output)
        linear_output = self.linear(dropout_output)

        return linear_output

Downloading:   0%|          | 0.00/416M [00:00<?, ?B/s]

Some weights of the model checkpoint at bert-base-cased were not used when initializing BertModel: ['cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.decoder.weight', 'cls.predictions.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [ ]:
model = BertClassifier().cuda()
model.bert.requires_grad_(False)

Some weights of the model checkpoint at bert-base-cased were not used when initializing BertModel: ['cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.decoder.weight', 'cls.predictions.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


BertModel(
  (embeddings): BertEmbeddings(
    (word_embeddings): Embedding(28996, 768, padding_idx=0)
    (position_embeddings): Embedding(512, 768)
    (token_type_embeddings): Embedding(2, 768)
    (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (encoder): BertEncoder(
    (layer): ModuleList(
      (0): BertLayer(
        (attention): BertAttention(
          (self): BertSelfAttention(
            (query): Linear(in_features=768, out_features=768, bias=True)
            (key): Linear(in_features=768, out_features=768, bias=True)
            (value): Linear(in_features=768, out_features=768, bias=True)
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (output): BertSelfOutput(
            (dense): Linear(in_features=768, out_features=768, bias=True)
            (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
            (dropout): Dropout(p=0.1, inplace=False)
          

In [ ]:
def save(model, args, epoch, optimizer, scheduler=None):
    # torch.save(model, args['model_save_path']+"_"+str(epoch))

    sv = {
      'model': model.state_dict(),
      'optimizer_state_dict': optimizer.state_dict(),
      'epoch': epoch
    }
    if scheduler is not None:
        sv['scheduler'] = scheduler.state_dict()

    torch.save(sv, args['model_save_path'])
    torch.save(sv, args['model_save_path']+"_"+str(epoch))

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

def train(model, train_dataloader, val_dataloader, epochs, optimizer):
  model.train()

  total_acc_train = 0
  total_loss_train = 0

  criterion = nn.CrossEntropyLoss()


  for epoch in range(epochs):
    num_egs = 0.0
    model.train()
    model.bert.requires_grad_(False)

    batch_bar = tqdm(total=len(train_loader), dynamic_ncols=True, position=0, leave=False, desc='Train')

    for batch_idx, (qs, length, modality) in enumerate(train_loader):
      num_egs += modality.shape[0]
      modality = modality.to(device)
      mask = qs['attention_mask'].to(device)
      input_id = qs['input_ids'].squeeze(1).to(device)

      # print(input_id)

      output = model(input_id, mask)
      
      batch_loss = criterion(output, modality)
      total_loss_train += batch_loss.item()
      
      acc = (output.argmax(dim=1) == modality).sum().item()
      total_acc_train += acc

      model.zero_grad()
      batch_loss.backward()
      optimizer.step()

      batch_bar.set_postfix(acc="{:.06f}".format(total_acc_train/num_egs))

      batch_bar.update()
    

    batch_bar.close()

    print(f'Train accuracy is total_acc_val {total_acc_train/num_egs}')

    total_loss_val = 0.0
    total_acc_val = 0.0

    with torch.no_grad():
      num_egs = 0.0
      model.eval()
      batch_bar = tqdm(total=len(val_dataloader), dynamic_ncols=True, position=0, leave=False, desc='Val')


      for batch_idx, (qs, length, modality) in enumerate(val_dataloader):
          num_egs += modality.shape[0]
          modality = modality.to(device)
          mask = qs['attention_mask'].to(device)
          input_id = qs['input_ids'].squeeze(1).to(device)

          output = model(input_id, mask)

          batch_loss = criterion(output, modality)
          total_loss_val += batch_loss.item()
          
          acc = (output.argmax(dim=1) == modality).sum().item()
          total_acc_val += acc

          batch_bar.set_postfix(acc="{:.06f}".format(total_acc_val/num_egs))

          batch_bar.update()

      batch_bar.close()

      save(model, args, epoch, optimizer, None)
    
      print(f'Val accuracy is total_acc_val {total_acc_val/num_egs}')


def test

In [ ]:
args = {
    'epochs': 20,
    'lr': 0.003,
    'model_save_path': '/content/drive/MyDrive/IDL-Project/Manymodal_qa_baseline/models/model.pt'
}

optimizer = torch.optim.Adam(model.parameters(), lr=args['lr'])

train(model, train_loader, val_loader, args['epochs'], optimizer)

Train accuracy is total_acc_val 0.9581678724908883


Val accuracy is total_acc_val 0.999597261377366


Train:  83%|████████▎ | 240/288 [34:19<06:51,  8.57s/it, acc=2.136003]

RuntimeError: ignored

In [ ]:
from torch import nn
from transformers import BertModel, BertConfig
from torch.nn.utils.rnn import pad_sequence, pack_padded_sequence, pad_packed_sequence

configuration = BertConfig('bert-base-cased')
bert = BertModel.from_pretrained('bert-base-cased').cuda().eval()

class BertClassifier(nn.Module):

    def __init__(self, dropout=0.5):

        super(BertClassifier, self).__init__()
        
        # print(configuration.hidden_size)
        self.dropout = nn.Dropout(dropout)
        self.reducer = nn.Linear(768,256)
        self.lstm = nn.LSTM(input_size=256, hidden_size=512,\
                            num_layers=2, bidirectional=True, dropout=.2)
        # self.relu = nn.ReLU()
        self.linear = nn.Sequential(
            nn.Linear(512*2,1024),
            nn.ReLU(),
            nn.Dropout(p=.2),
            nn.Linear(1024,512),
            nn.ReLU(),
            nn.Dropout(p=.2),
            nn.Linear(512,2)
        )

    def forward(self, seq_output, lens):
        batch_size = seq_output[0].shape[0]
        # print(batch_size)
        # with torch.no_grad():
        #    = self.
        # print(seq_output[0].shape)
        dropout_output = self.dropout(self.reducer(seq_output[0]))
        packed_input = pack_padded_sequence(dropout_output, lengths=lens, batch_first=True, enforce_sorted=False)
        out1, (out2, out3) = self.lstm(packed_input)
        out, lengths  = pad_packed_sequence(out1,batch_first=True)
        # print(out.shape, lens,lengths,out[:,-1,:].view(batch_size,-1).shape)
        final_layer = self.linear(out[:,-1,:].view(batch_size,-1))

        return final_layer

model = BertClassifier().cuda()
try:
  import torchsummaryX
except:
  !pip install torchsummaryX
  import torchsummaryX
from torchsummaryX import summary

x = next(iter(train_loader))
with torch.no_grad():
  seq_output =  bert(input_ids= x[0]['input_ids'].squeeze().cuda(), attention_mask=x[0]['attention_mask'].cuda(),return_dict=False)
summary(model, seq_output, x[1])

In [ ]:
import torch
import numpy as np
import torch.nn as nn
import torch.nn.functional as F
from torchsummaryX import summary
from torch.utils.data import Dataset, DataLoader
from torch.nn.utils.rnn import pad_sequence, pack_padded_sequence, pad_packed_sequence
import torch.optim as optim


optimizer = optim.Adam(model.parameters(),lr=1e-3, weight_decay=4e-6)
criterion = nn.CrossEntropyLoss()

In [ ]:
from tqdm import tqdm
def validate():
  model.eval()
  batch_bar = tqdm(total=len(val_loader), dynamic_ncols=True, position=0, leave=False, desc='Val')
  num_correct = 0
  total_loss = 0.
  for i , (q, lens, target) in enumerate(val_loader):
    # print(q['input_ids'].shape)
    input_id = q['input_ids'].squeeze().cuda()
    mask = q['attention_mask'].cuda()
    with torch.no_grad():
      seq_output =  bert(input_ids= input_id, attention_mask=mask,return_dict=False)
    output = model(seq_output,lens)
    loss = criterion(output, target.cuda())
   
    num_correct += int((torch.argmax(output, axis=1) == target.cuda()).sum())

    batch_bar.set_postfix(acc="{:.04f}%".format(100 * num_correct / ((i + 1) * 128)))
    batch_bar.update()
  val_loss = total_loss / len(val_loader)   
  batch_bar.close()
  print("\n")
  print("Validation: {:.04f}%".format(100 * num_correct / (len(val_loader)*128)))
  return num_correct/(len(val_loader)*128) *100, val_loss

for epoch in range(50):
  batch_bar = tqdm(total=len(train_loader), dynamic_ncols=True, leave=False, position=0, desc='Train') 
  num_correct = 0
  total_loss = 0
  model.train()
  for i , (q, lens, target) in enumerate(train_loader):
    # print(q['input_ids'].shape)
    input_id = q['input_ids'].squeeze().cuda()
    mask = q['attention_mask'].cuda()
    with torch.no_grad():
      seq_output =  bert(input_ids= input_id, attention_mask=mask,return_dict=False)
    output = model(seq_output,lens)
    loss = criterion(output, target.cuda())

    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    num_correct += int((torch.argmax(output, axis=1) == target.cuda()).sum())
    total_loss += float(loss)
    batch_bar.set_postfix(
              acc="{:.04f}%".format(100 * num_correct / ((i + 1) * 128)),
              loss="{:.04f}".format(float(total_loss / (i + 1))),
              num_correct=num_correct,
              lr="{:.04f}".format(float(optimizer.param_groups[0]['lr'])))
    batch_bar.update() 

  batch_bar.close()
  acc = 100 * num_correct / (len(train_loader) * 128)
  tr_loss = float(total_loss / len(train_loader))

  print("\n")
  print("Epoch {}/{}: Train Acc {:.04f}%, Train Loss {:.04f}".format(
        epoch + 1,
        50,
        acc,
        tr_loss))
  val_acc,val_loss = validate()
  print(f"valid Acc: {val_acc}")



